# Regressão Linear Multivariada com NumPy

Autor: Rodolfo Marinho

Email: rodolfomarinho (at) copin (dot) ufcg (dot) edu (dot) br

In [1]:
import numpy as np
import math
import time

### Versão não vetorizada
Função para calcular o MSE (Mean Squared Error):

$MSE(\hat{w}) = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i (x_i))^2$

In [2]:
# y = mx + b
# m is slope, b is y-intercept
def compute_hypothesis(b, m, X):
    h_x = b
    for i in range(m.size):
        h_x += X[i] * m[i]
    return h_x

def compute_mse(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        X = points[i, :-1]
        y = points[i, -1]
        totalError += (y - compute_hypothesis(b, m, X))**2
    return totalError / float(len(points))

Função para fazer uma atualização dos parâmetros no Gradiente Descendente:

$w_0 = w_0 + 2\alpha\sum_{i=1}^N (y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$

$w_1 = w_1 + 2\alpha\sum_{i=1}^N x_{i_1}(y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$

...

$w_n = w_n + 2\alpha\sum_{i=1}^N x_{i_n}(y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$

In [3]:
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = np.zeros(m_current.size)
    for i in range(0, len(points)):
        X = points[i, :-1]
        y = points[i, -1]
        
        h_x = compute_hypothesis(b_current, m_current, X)
        b_gradient += y - h_x
        for j in range(m_gradient.size):
            m_gradient[j] += X[j] * (y - h_x)
    new_b = b_current + (2 * learningRate * b_gradient)
    new_m = np.add(m_current, 2 * learningRate * m_gradient)
    return [new_b, new_m, b_gradient, m_gradient]

$||\mathbf{w}||_2 = \sqrt{\sum_{j=1}^D w_j^2}$

In [4]:
def norm_2(x):
    c=0
    for i in range(len(x)):
        c += x[i]**2
    return math.sqrt(c)

Função para iterar sobre o gradiente descendente até convergência.

In [5]:
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, epsilon):
    b = starting_b
    m = starting_m
    grad = np.array([np.inf]*(starting_m.size + 1))
    i = 0
    while (norm_2(grad)>=epsilon):
        b, m, b_gradient, m_gradient = step_gradient(b, m, points, learning_rate)
        grad = [b_gradient]
        grad.extend(m_gradient)
        grad = np.array(grad)
        #print("Norm2:",norm_2(grad))
        if i % 1000 == 0:
            #print(norm_2(grad))
            print("MSE na iteração {0} é de {1}".format(i,compute_mse(b,m,points)))
        i+= 1
    return [b, m]

In [6]:
points = np.genfromtxt("sample_treino.csv", delimiter=",")
learning_rate = 0.000035
initial_b = 0 # initial y-intercept guess
initial_m = np.zeros(points[0].size-1) # initial slope guess
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_mse(initial_b, initial_m, points)))
print("Running...")
tic = time.time()
[b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com b = {0}, m = {1}, erro = {2}".format(b, m, compute_mse(b, m, points)))
print("Versão não vetorizada rodou em: " + str(1000*(toc-tic)) + "ms")

Starting gradient descent at b = 0, m = [0. 0. 0. 0. 0.], error = 54.47995385562646
Running...
MSE na iteração 0 é de 33.302186462253474
MSE na iteração 1000 é de 0.4300627860129182
MSE na iteração 2000 é de 0.42848689972774284
MSE na iteração 3000 é de 0.42707721580379643
MSE na iteração 4000 é de 0.42578343897705406
MSE na iteração 5000 é de 0.42459600894933835
MSE na iteração 6000 é de 0.42350618412280805
MSE na iteração 7000 é de 0.42250594147706977
MSE na iteração 8000 é de 0.42158791747358043
MSE na iteração 9000 é de 0.4207453538470714
MSE na iteração 10000 é de 0.419972047852873
MSE na iteração 11000 é de 0.4192623066038457
MSE na iteração 12000 é de 0.41861090516075666
MSE na iteração 13000 é de 0.4180130480675691
MSE na iteração 14000 é de 0.4174643340484845
MSE na iteração 15000 é de 0.4169607236068418
MSE na iteração 16000 é de 0.4164985092873406
MSE na iteração 17000 é de 0.41607428838267635
Gradiente descendente convergiu com b = 0.9237624803254038, m = [0.11762623 0.0836

### Versão Vetorizada

$MSE(\hat{w})=\frac{1}{N}(y-\hat{\mathbf{w}}^T\mathbf{x})^T(y-\hat{\mathbf{w}}^T\mathbf{x})$

In [7]:
def compute_mse_vectorized(w,X,Y):
    res = Y - np.dot(X,w)
    totalError = np.dot(res.T,res)
    return totalError / float(len(Y))

In [19]:
def step_gradient_vectorized(w_current,X,Y,learningRate):
    res = Y - np.dot(X,w_current)
    b_gradient = np.sum(res)
    m_gradient = np.matmul(np.transpose(X),res)
    new_w = np.array([(w_current[0] + (2 * learningRate * b_gradient))] +
             [np.add(np.array(w_current[1:]), (2 * learningRate * m_gradient[1:]))])
    return [new_w,b_gradient,m_gradient]
    

In [10]:
def gradient_descent_runner_vectorized(starting_w, X,Y, learning_rate, epsilon):
    w = starting_w
    grad = np.array([np.inf]*w.size)
    i = 0
    while (np.linalg.norm(grad)>=epsilon):
        w,b_gradient,m_gradient = step_gradient_vectorized(w, X, Y, learning_rate)
        grad = [b_gradient]
        grad.extend(m_gradient)
        grad = np.array(grad)
        #print(np.linalg.norm(grad))
        if i % 1000 == 0:
            print("MSE na iteração {0} é de {1}".format(i,compute_mse_vectorized(w, X, Y)))
        i+= 1
    return w

In [20]:
points = np.genfromtxt("sample_treino.csv", delimiter=",")
points = np.c_[np.ones(len(points)),points]
X = points[:,:-1]
Y = points[:,-1]
init_w = np.zeros((X[0].size))
learning_rate = 0.000035
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at W = {0}, error = {1}".format(init_w, compute_mse_vectorized(init_w, X,Y)))
print("Running...")
tic = time.time()
w = gradient_descent_runner_vectorized(init_w, X,Y, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, error = {2}".format(w[0], w[1], compute_mse_vectorized(w,X,Y)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")


Starting gradient descent at W = [0. 0. 0. 0. 0. 0.], error = 54.47995385562645
Running...


ValueError: setting an array element with a sequence.